In [330]:
# import libraries
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import os.path
import time
# from distanceMatrix import *
# define functions
%run distanceMatrix.py
%run generateClients.py

# instance size
N = 30
# number of trucks available
K = 30
# maximum tour time
T = int(600)  # 4.5h = 270min
# verbosity of report
verbose = 0

# load the data and files
clientsFile = 'clients'+str(N)+'.csv'
# 0     =7am
# 600   =5pm
# noon  =300
# 1pm   =360
# 4pm   =540
# generate clients if necesary
if not os.path.isfile(clientsFile):
    print('clientsFile does not exist')
    generateClients(N,'belgian-cities-geocoded.csv')
clients = pd.read_csv(clientsFile)
wpfsFile = 'WPF.csv'
citiesFile = 'belgian-cities-geocoded.csv'
cities = pd.read_csv(citiesFile)
distanceMatrixFile = 'distanceMatrix'+str(N)+'.csv'
wpfs = pd.read_csv(wpfsFile)

# get the matrix and save it if necessary
if not os.path.isfile(distanceMatrixFile):
    print('distanceMatrix does not exist')
    distanceMatrix = createDistanceMatrix(clientsFile, citiesFile, wpfsFile)
    pd.DataFrame(distanceMatrix).to_csv(distanceMatrixFile, index=False)
distanceMatrix = pd.read_csv(distanceMatrixFile).to_numpy()


# get coordinates
coords = [(50.9338827, 4.5605498)]  # depot
for client in clients['Place']:
    idx = cities.index[cities['name'] == client].tolist()[0]
    coords.append((cities.iloc[idx]['lat'], cities.iloc[idx]['lng']))

# get timewindows
timeWindows = [(0, 600)]  # depot
for id in clients['ClientID']:
    opening = clients['opening'][id-1]
    closing = clients['closing'][id-1]
    opening = int(opening)
    closing = int(closing)
    timeWindow = (opening, closing)
    timeWindows.append(timeWindow)


In [333]:
# build the model and solve it
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = distanceMatrix
    data['time_windows'] = timeWindows
    data['num_vehicles'] = K
    data['depot'] = 0
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    total_length = 0
    # total_waiting = 0
    total_waiting_depot=0
    total_waiting_clients=0
    z=0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        waiting_depot=0
        waiting_clients=0
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            if index*solution.Min(time_var) != 0:
                if index > N:
                    waiting_depot+=solution.Min(time_var)
            plan_output += '{0} ({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))   
            index = solution.Value(routing.NextVar(index))
            waiting_clients += solution.Max(time_var)-solution.Min(time_var)
        time_var = time_dimension.CumulVar(index)
        tour_length = solution.Min(time_var) - (waiting_depot+waiting_clients)
        tour_time = solution.Min(time_var)
        plan_output += '{0} ({1},{2})\n'.format(
            manager.IndexToNode(index),
            solution.Min(time_var),
            solution.Max(time_var))
        if tour_time > 0:
            z+=1
            plan_output += 'Elapsed time (min): {}\n'.format(
                tour_time)
            plan_output += 'Tour length (min): {}\n'.format(
                tour_length)
            plan_output += 'Waiting time: {} @ depot + {} @ clients\n'.format(
                waiting_depot,
                waiting_clients)
        # if verbose:
            print(plan_output)
        total_waiting_clients += waiting_clients
        total_waiting_depot += waiting_depot
        # total_waiting = total_waiting_clients+total_waiting_depot
        total_time += solution.Min(time_var)
        total_length += tour_length
    print(
        f'Objective: {solution.ObjectiveValue()}',
        f'  |  Vehicles in use: {z}')
    print(f'Sum elapsed times (min): {total_time}')
    print(f'Sum non-waiting times (min): {total_length}')
    print('Sum waiting times (min): {} @ depot + {} @ clients'.format(
        total_waiting_depot,
        total_waiting_clients
    ))

def get_cumul_data(solution, routing, dimension):
  """Get cumulative data from a dimension and store it in an array."""
  # Returns an array cumul_data whose i,j entry contains the minimum and
  # maximum of CumulVar for the dimension at the jth node on route :
  # - cumul_data[i][j][0] is the minimum.
  # - cumul_data[i][j][1] is the maximum.

  cumul_data = []
  for route_nbr in range(routing.vehicles()):
    route_data = []
    index = routing.Start(route_nbr)
    dim_var = dimension.CumulVar(index)
    route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      dim_var = dimension.CumulVar(index)
      route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    cumul_data.append(route_data)
  return cumul_data

def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

In [334]:
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback.
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Time Windows constraint.
time = 'Time'
routing.AddDimension(
    transit_callback_index,
    30,  # allow waiting time @clients
    600,  # maximum time available per vehicle; 10hours: 7am to 5pm
    False,  # Don't force start cumul to zero.
    time)

time_dimension = routing.GetDimensionOrDie(time)

## set maximum tour time for vehicles
for vehicle_id in range(data['num_vehicles']):
    time_dimension.SetSpanUpperBoundForVehicle(600,vehicle_id)
    # routing.SetFixedCostOfVehicle(500, vehicle_id)
    # time_dimension.SetCumulVarSoftLowerBound(vehicle_id, 300, 1)

# Add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data['time_windows']):
    if location_idx == data['depot']:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
# Add time window constraints for each vehicle start node.
depot_idx = data['depot']
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data['time_windows'][depot_idx][0],
        data['time_windows'][depot_idx][1])

# Instantiate route start and end times to produce feasible times.
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
# setting the local search metaheuristic
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.seconds = 10
search_parameters.log_search = True

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)
    # get solution windows
    solution_windows=get_cumul_data(solution,routing,time_dimension)
    solution_routes=get_routes(solution,routing,manager)

else:
    print("Solver status: ",routing.status())
    print("0 	ROUTING_NOT_SOLVED: Problem not solved yet.\n1 	ROUTING_SUCCESS: Problem solved successfully.\n2 	ROUTING_FAIL: No solution found to the problem.\n3 	ROUTING_FAIL_TIMEOUT: Time limit reached before finding a solution.\n4 	ROUTING_INVALID: Model, model parameters, or flags are not valid.  ")  

Route for vehicle 0:
0 (0,0) -> 22 (44,44) -> 27 (162,162) -> 12 (250,250) -> 1 (366,366) -> 7 (444,444) -> 2 (537,537) -> 0 (552,552)
Elapsed time (min): 552
Tour length (min): 552
Waiting time: 0 @ depot + 0 @ clients

Route for vehicle 19:
0 (0,0) -> 16 (262,262) -> 8 (492,492) -> 0 (558,558)
Elapsed time (min): 558
Tour length (min): 558
Waiting time: 0 @ depot + 0 @ clients

Route for vehicle 21:
0 (0,0) -> 19 (206,206) -> 11 (398,398) -> 0 (450,450)
Elapsed time (min): 450
Tour length (min): 450
Waiting time: 0 @ depot + 0 @ clients

Route for vehicle 22:
0 (0,0) -> 29 (173,173) -> 28 (326,326) -> 13 (498,498) -> 0 (565,565)
Elapsed time (min): 565
Tour length (min): 565
Waiting time: 0 @ depot + 0 @ clients

Route for vehicle 23:
0 (0,0) -> 17 (233,248) -> 23 (440,440) -> 24 (489,489) -> 0 (580,580)
Elapsed time (min): 580
Tour length (min): 565
Waiting time: 0 @ depot + 15 @ clients

Route for vehicle 24:
0 (0,0) -> 26 (121,121) -> 4 (268,268) -> 0 (308,308)
Elapsed time (min):

In [ ]:
# Map the solution
import folium

# list of colors to be used in the map
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred','black', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray','red', 'blue', 'green', 'purple', 'orange', 'darkred','lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# zoom in on Belgium
map = folium.Map(location=[50.5,4], zoom_start = 8)

# # route 0 example
postProcessedRoutes = [
    # [ # route 0
    #     (50.9338827, 4.5605498),#depot
    #     (50.98707, 5.36716),#zonhoven
    #     (51.0258791, 4.4775365),#mechelen
    #     (50.98707, 5.36716),#zonhoven
    #     (50.9215166, 5.3447405),#hasselt
    #     (51.2198771, 4.4011356),#antwerp
    #     (50.9215166, 5.3447405),#hasselt
    #     (51.2198771, 4.4011356),#antwerp
    #     (50.9215166, 5.3447405),#hasselt
    #     (50.9338827, 4.5605498),#depot
    # ],
    # [], # route 1
    [ # route 2
        (50.9338827, 4.5605498),# depot
        (50.8492265, 2.8779465),# ieper
        (51.2093488, 3.2247013),# brugge
        (50.8194894, 3.2577076),# kortjrijk
        (51.0678307, 3.7290914),# gent
        (50.8427501, 4.3515499),# brussel
        (51.0258791, 4.4775365),# mechelen
        (51.0953745, 4.5052038),# duffel
        (50.9338827, 4.5605498),# depot
    ]
]

for idx,postProcessedRoute in enumerate(postProcessedRoutes):
    if len(postProcessedRoute)>0:
        folium.PolyLine(postProcessedRoute,weight=len(postProcessedRoute),color=colors[3],popup='route'+str(2)).add_to(map)
        


# visualize the routes
# solution_routes_nonEmpty = [route for route in solution_routes if len(route) > 2]
# for idx,route in enumerate(solution_routes_nonEmpty):
#     route_coords=[]
#     for node in route:
#         route_coords.append(coords[node])
#     folium.PolyLine(route_coords,weight=len(route),color=colors[idx],popup=route).add_to(map)

    

# visualize the nodes
for idx,coord in enumerate(coords):
    if(idx == 0):
        folium.Marker(
            location=[coord[0], coord[1]],
            popup='#0\nKampenhout',
            icon=folium.Icon(color='black',icon="home"),
        ).add_to(map)
    else:
        icon_colors = ['green','lightblue','cadetblue','orange','lightgray','red']
        folium.Marker(
            location=[coord[0], coord[1]],
            popup='#\n'+str(clients['ClientID'][idx-1])+' '+clients['Place'][idx-1]+'\n'+str(clients['ActionType'][idx-1])+'\n'+str(coord),
            icon=folium.Icon(color=icon_colors[clients['ActionType'][idx-1]-1], icon='', prefix='fa')
        ).add_to(map)

# visualize the WPFs
for idx,wpf in enumerate(wpfs['Place']):
    folium.Marker(
        location=[wpfs.iloc[idx].lat, wpfs.iloc[idx].lon],
        popup=wpfs.iloc[idx].Place+'\n'+str(wpfs.iloc[idx].lat)+', '+str(wpfs.iloc[idx].lon),
        icon=folium.Icon(color='black',icon="trash",prefix='fa'),
    ).add_to(map)

map.save('routing'+str(N)+'.html')
map


In [ ]:
a=0
b=4
distanceKmFromCoord(
    # 50.98707, 5.36716, #zonhoven
    # 50.9215166, 5.3447405, #hasselt
    50.9338827, 4.5605498, #depot - kampenhout
    # 51.2198771, 4.4011356, # antwerp
    # 51.0258791, 4.4775365, # mechelen
    # 51.191087, 5.1170647, # mol
    # 50.9806151, 4.827381 # aarschot
    # 51.0731945, 2.6680059, # veurne
    # 51.2093488, 3.2247013, # brugge
    # 51.08608, 4.36633, #boom
    # 50.8492265, 2.8779465, #ieper
    # 50.8194894, 3.2577076, #kortjrijk
    # 51.0678307, 3.7290914, #gent
    # 50.8427501, 4.3515499, #brussel
    # 51.0953745, 4.5052038, # duffel
    # 50.693646, 5.5638281, # liers
    # 51.0010401, 5.0852169, # dies
    # 51.1987114, 3.8119772, # zelzate
    # 50.668081, 4.6118324, # louvain
    # 51.16257, 4.99084, # geel
    # 50.81057, 4.93622, # tienen
    # 50.8815197, 4.6967578, #leuven
    # 50.64786, 5.54357, # glein
    # 50.84109, 4.83471, # boutersem
    # 51.1768785, 4.835648, # herentals
    # 51.0614235, 4.5055555, # sint-katelijne
    # 50.9356235, 4.3785596, #grimbergen
    # 50.8795024, 4.4826212, #zaventem
    # 51.0678307, 3.7290914, # gent
    49.9750202,5.3892229, # Bras
)/50*60

In [ ]:
# %run generateClients.py
# generateClients(200,'belgian-cities-geocoded.csv')

In [ ]:
# %run distanceMatrix.py
# createDistanceMatrix(clientsFile, citiesFile, wpfsFile)